In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from statsmodels.tsa.seasonal import seasonal_decompose

from src.plot import plot_single_bath, plot_all_params, save_individual_run, save_bath, plot_conductivity_fft
from src.load_data import load_sheet, clean_sheet_with_label

%load_ext autoreload
%matplotlib
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using matplotlib backend: module://matplotlib_inline.backend_inline


In [31]:
import os

print(os.getcwd())

c:\Users\kittyoomm\Desktop\Work@MTEC\PlateMon_data_analytics


In [32]:
internal_df, sheet1 = load_sheet("Electroplate Experiments Data JUN_JUL", "vary_internal_table")
internal_df = clean_sheet_with_label(internal_df)
# two_side_plate, sheet2 = load_sheet("Electroplate Experiments Data JUN_JUL", "2 side Experimental Result")
# two_side_plate = clean_sheet_with_label(two_side_plate)
august_df, sheet3 = load_sheet("Electroplating Experiments Data August", "Sheet1")
august_df = clean_sheet_with_label(august_df)
september_df, sheet4 = load_sheet("Electroplating Experiments Data September", "Sheet1")
september_df =clean_sheet_with_label(september_df)

internal_df[internal_df['bath_id']=='Bath_3']
combined_df = pd.concat([internal_df, august_df, september_df])

In [33]:
jul_22_3 = combined_df[combined_df['run_id']=='JUL_22_3'].copy()
print(f'Anomaly C type: {jul_22_3['Anomaly C'].unique()}')
print(f'Anomaly P type: {jul_22_3['Anomaly P'].unique()}')
print(f'Anomaly T type: {jul_22_3['Anomaly T'].unique()}')
print(f'Anomaly V type: {jul_22_3['Anomaly V'].unique()}')

Anomaly C type: ['C3']
Anomaly P type: ['N']
Anomaly T type: ['N']
Anomaly V type: ['N']


In [34]:
SENSOR_FEATURES = ["conductivity", "temperature", "pH",	"voltage"]
TRAIN_RUNS = ['JUL_22_1','JUL_22_2', 'JUL_22_3', 'JUL_22_4', 'JUL_22_5', 'JUL_22_6','JUL_22_7', 'JUL_22_8', 'JUL_22_9', 
			  'JUL_23_1', 'JUL_23_2', 'JUL_23_3', 'JUL_23_4', 'JUL_23_5']
TEST_RUNS = ['JUL_24_1', 'JUL_24_2',
       'JUL_24_3', 'JUL_24_4', 'JUL_24_5', 'JUL_24_6', 'JUL_24_7',
       'JUL_24_8', 'JUL_24_9', 'JUL_24_10']

#NORMAL_RUNS = ['JUL_22_4', 'JUL_22_5', 'JUL_22_7']
#NORMAL_CONDUCTIVITY_RUNS = ['JUL_22_1','JUL_22_2', 'JUL_22_4', 'JUL_22_5', 'JUL_22_7']

In [35]:
# Function to separate normal and anomalous data based on specified anomaly label
def get_noramal_data(df: pd.DataFrame, runs: list, parameter: str) -> pd.DataFrame:
    NORMAL_RUNS = []
    VALID_RUNS = []
    for run in runs:
        checked_df = df[df['run_id'] == run]
    
        if (checked_df[parameter].unique() == ['N']).all():# and (df['Anomaly P'].unique() == ['N']).all() and (df['Anomaly T'].unique() == ['N']).all() and (df['Anomaly V'].unique() == ['N']).all():
            if run in NORMAL_RUNS:
                continue
            else:
                NORMAL_RUNS.append(run)
        else:
            VALID_RUNS.append(run)

    print("Normal runs:", NORMAL_RUNS)
    print("Valid runs:", VALID_RUNS)
    
    train_df = df[df['run_id'].isin(NORMAL_RUNS)].copy()
    train_df = train_df.reset_index(drop=True)
    train_df['index'] = train_df.index
    
    valid_df = df[df['run_id'].isin(VALID_RUNS)].copy()
    valid_df = valid_df.reset_index(drop=True)
    valid_df['index'] = valid_df.index
    
    return train_df, valid_df

In [36]:
# Seperating normal and anomalous data based on 'Anomaly C' label --> used as train and validation datasets, respectively
train_c_df, valid_c_df = get_noramal_data(combined_df, TRAIN_RUNS, 'Anomaly C')

Normal runs: ['JUL_22_1', 'JUL_22_2', 'JUL_22_4', 'JUL_22_5', 'JUL_22_7']
Valid runs: ['JUL_22_3', 'JUL_22_6', 'JUL_22_8', 'JUL_22_9', 'JUL_23_1', 'JUL_23_2', 'JUL_23_3', 'JUL_23_4', 'JUL_23_5']


In [37]:
# Checking anomaly label types in each dataset
print(f'----------------------df------------------------')
print(f'Anomaly C type: {combined_df['Anomaly C'].unique()}')
print(f'Anomaly P type: {combined_df['Anomaly P'].unique()}')
print(f'Anomaly T type: {combined_df['Anomaly T'].unique()}')
print(f'Anomaly V type: {combined_df['Anomaly V'].unique()}')
print(f'-------------------Train df---------------------')
print(f'Anomaly C type: {train_c_df['Anomaly C'].unique()}')
print(f'Anomaly P type: {train_c_df['Anomaly P'].unique()}')
print(f'Anomaly T type: {train_c_df['Anomaly T'].unique()}')
print(f'Anomaly V type: {train_c_df['Anomaly V'].unique()}')
print(f'-----------------Validation df------------------')
print(f'Anomaly C type: {valid_c_df['Anomaly C'].unique()}')
print(f'Anomaly P type: {valid_c_df['Anomaly P'].unique()}')
print(f'Anomaly T type: {valid_c_df['Anomaly T'].unique()}')
print(f'Anomaly V type: {valid_c_df['Anomaly V'].unique()}')

----------------------df------------------------
Anomaly C type: ['' 'N' 'C3' 'C1' 'C2' None]
Anomaly P type: ['' 'N' None]
Anomaly T type: ['' 'N' None]
Anomaly V type: ['' 'N' 'V1' 'V2' None]
-------------------Train df---------------------
Anomaly C type: ['N']
Anomaly P type: ['N']
Anomaly T type: ['N']
Anomaly V type: ['N' 'V1' 'V2']
-----------------Validation df------------------
Anomaly C type: ['C3' 'N' 'C1' 'C2']
Anomaly P type: ['N']
Anomaly T type: ['N']
Anomaly V type: ['N' 'V1' 'V2']


In [38]:
# Saving processed datasets to Excel files
train_c_df = train_c_df[['run_id', 'conductivity', 'index', 'timestamp']]
train_c_df.to_excel('Datasets/train_c_data.xlsx', index=False)

valid_c_df['anomaly_label'] = valid_c_df['Anomaly C'].apply(lambda x: 0 if x == 'N' else 1) # Creating binary anomaly label
valid_c_df = valid_c_df[['run_id', 'conductivity', 'index', 'timestamp', 'anomaly_label']]
valid_c_df.to_excel('Datasets/valid_c_data.xlsx', index=False)

---------- ^^^ ---------- Used Code ---------- ^^^ ----------

In [7]:
# Assuming your time column is named 'timestamp' and is not yet the index
filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp']) # Convert to datetime objects
filtered_df = filtered_df.set_index('timestamp') # Set it as the index

In [8]:
# --- 1. Create a dummy DataFrame for demonstration (skip this if you have your data) ---
# Your original data has 600 points collected every 2 seconds
start_time = pd.to_datetime('2025-01-01 00:00:00')
end_time = start_time + pd.Timedelta(seconds=(len(filtered_df)-1) * 2) # Total duration is 1198 seconds, or 19 min 58 sec
index = pd.date_range(start=start_time, end=end_time, freq='2S')
filtered_df = filtered_df.set_index(index) # Set it as the index
filtered_df
# -------------------------------------------------------------------------------------
resampled_df = filtered_df.resample('2s').first()
resampled_df

C:\Users\kittyoomm\AppData\Local\Temp\ipykernel_11924\1961443209.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  index = pd.date_range(start=start_time, end=end_time, freq='2S')


,run_id,time_total,conductivity,Anomaly C,pH,Anomaly P,temperature,Anomaly T,voltage,Anomaly V
2025-01-01 00:00:00,JUL_22_1,2,56.32,N,4.02,N,49.70,N,3.639,N
2025-01-01 00:00:02,JUL_22_1,4,56.32,N,4.02,N,49.70,N,3.705,N
2025-01-01 00:00:04,JUL_22_1,6,56.17,N,4.02,N,49.71,N,3.702,N
2025-01-01 00:00:06,JUL_22_1,8,56.17,N,4.02,N,49.68,N,3.673,N
2025-01-01 00:00:08,JUL_22_1,10,56.17,N,4.02,N,49.69,N,3.668,N
...,...,...,...,...,...,...,...,...,...,...
2025-01-01 04:58:46,JUL_24_3,20318,56.80,N,4.25,N,49.99,N,3.459,
2025-01-01 04:58:48,JUL_24_3,20320,57.01,C3,4.25,N,49.98,N,3.461,
2025-01-01 04:58:50,JUL_24_3,20322,56.80,N,4.25,N,49.97,N,3.466,
2025-01-01 04:58:52,JUL_24_3,20324,56.80,N,4.25,N,49.98,N,3.463,
